In [202]:
class HiddenNumber:
    
    def __init__(self,p):
        #gerar o segredo
        self.secret = ZZ.random_element(p)
        print 'The Secret is: '; print self.secret
    
    '''
    Aproximação dos k bits mais significativos do seu 
    argumento módulo p
    '''
    def msb(self,k,p,xi):
        val = ZZ(Mod(xi*self.secret,p))
        bits = val.digits(2)
        bits.reverse()
        final = ''
        if len(bits) < k:
            final = ''.join(str(e) for e in bits)
        else :
            for i in range(0,k):
                final += str(bits[i])
        return final

    def compare(self,calc):
        if calc == self.secret:
            return True
        else:
            return False
    


In [196]:
# l inteiros positivos no intervalo {0,p-1}
def generate_xvector(l,p):
    x = []
    for i in range(0,l):
        x.append(ZZ.random_element(p))
    return x

'''
xv -> x vector
k -> número > 0
p -> primo

vetor de l inteiros positivos que verificarem as igualdades
ui = msb(s*xi)
'''
def generate_uvector(xv,k,p,hnp):
    u = []
    for xi in xv:
        ui = hnp.msb(k,p,xi)
        #binario para decimal
        ui = ZZ(int(ui,2))
        u.append(ui)
    return u
    


In [206]:
import sage.modules.free_module_integer as fmi
import numpy as np

class BV:
    
    def __init__(self,uv,xv,p,k,l):
        
        #gerar lambda, matriz L, target T e reticulado
        self.comp_lambda = 2^(k+1)
        self.L = self.comp_lambda * p * matrix.identity(l)
        self.L = self.L.transpose()
        self.L = self.L.insert_row(l,zero_vector(l))
        self.L = self.L.transpose()
        x = [self.comp_lambda * i for i in xv]
        x.append(1)
        self.L = self.L.insert_row(l,x)
        
        #gerar matriz target
        self.target = [self.comp_lambda * i for i in uv]
        self.target.append(0)
        self.target = matrix(self.target)
        self.retic = fmi.IntegerLattice(self.L)
    
    def CVP_resolve(self,x,p,l):
        L = matrix(self.retic.reduced_basis)
        t = matrix(1,l+1,list(-self.target))
        zero = matrix(l+1,1,[0]*(l+1))
        M = matrix(1,1,[p**2])
        L1 = block_matrix(2,2,[[L,zero],[t,M]])
        #base reduzida do reticulado de base L1
        Lred = fmi.IntegerLattice(L1).reduced_basis
        err1 = np.array(Lred[l+1][:-1])
        y1 = err1 + self.target
        # última componente do vetor resultante
        return y1[0][l]
        

In [203]:
l = 2^7
k = 64
p = 2^64
xv = generate_xvector(l,p)
hnp = HiddenNumber(p)
uv = generate_uvector(xv,k,p,hnp)
bv = BV(uv,xv,p,k,l)
calc = bv.CVP_resolve(xv,p,l)
if hnp.compare(calc):
    print 'True'
else:
    print 'False'

The Secret is: 
642972440413110417
[[78081381039019789320372658584453906432
  213057438319348056718048279110485016576
  615780366106072517331914356339811287040
  441204526100468420159248346405052350464
  153547619660838806863571587896690343936
  109777465443236235661413969835654643712
  98075563941155851879982565815188717568
  36193179169284926420979343930010107904
  371551857281818570075667395581961043968
  633636636521860177069510776103509164032
  516979728640956152532819355922096717824
  560298405849983172282858919217088954368
  501042441756703527855688079599990734848
  668302530617607756099154863397099536384
  553917219053754273088031391010047131648
  231185618326865815044273869150297784320
  234518485609399043945028872821480095744
  128320611784459331671672757185352302592
  463660220490181626862802115284961329152
  518145529775140485692578593882649919488
  541049246196387323078766150609717428224
  93249354181683202859753535487569035264
  554802717682484459370524814501595840512
  4

In [171]:
import sage.modules.free_module_integer as fmi
import numpy as np

l=2^2
k=16
p=7
x = generate_xvector(l,p)
sec = genSecret(p)
u = generate_uvector(sec,x,k,p)

#gerar matriz L
comp_lambda = 2^(k+1)
L = comp_lambda * p * matrix.identity(l)
L = L.transpose()
L = L.insert_row(l,zero_vector(l))
L = L.transpose()
x = [comp_lambda * i for i in x]
x.append(1)
L = L.insert_row(l,x)


#gerar matriz target
target = [comp_lambda * i for i in u]
target.append(0)
target = matrix(target)
retic = fmi.IntegerLattice(L)

L = matrix(retic.reduced_basis)
t = matrix(1,l+1,list(-target))
zero = matrix(l+1,1,[0]*(l+1))
M = matrix(1,1,p**2)
print(M)
L1 = block_matrix(2,2,[[L,zero],[t,M]])
print(L1)
ret = fmi.IntegerLattice(L1).reduced_basis
err = np.array(ret[l+1][:-1])
print(err)
y1 = err + target


The Secret is: 
2
[49]
[      0       0       0       0      -7|      0]
[ 393216 -131072 -262144  131072       3|      0]
[ 131072  262144 -393216 -262144       1|      0]
[ 524288  131072  262144 -131072      -3|      0]
[-262144  393216 -131072  524288      -2|      0]
[---------------------------------------+-------]
[-262144 -524288 -131072 -393216       0|     49]
[-262144  393216 -131072  524288      -2]
[[     0 917504      0 917504     -2]]
-2
